In [358]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.calibration import CalibratedClassifierCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,cross_val_score,KFold,ShuffleSplit,StratifiedKFold
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier,BaggingClassifier,ExtraTreesClassifier
from sklearn.metrics import r2_score,roc_auc_score,accuracy_score,confusion_matrix
import matplotlib.pyplot as plt
from mlxtend.classifier import StackingCVClassifier
import lightgbm as lgb 

%matplotlib inline
import seaborn as sns
import xgboost as xgb
from scipy.stats import pearsonr

In [2]:
pd.set_option('max_columns',100)

In [521]:
train=pd.read_csv('/Users/shashank/Downloads/Societe2/train.csv')
test=pd.read_csv('/Users/shashank/Downloads/Societe2/test.csv')
sample=pd.read_csv('/Users/shashank/Downloads/Societe2/sample_submissions.csv')

In [522]:
# ids=test['transaction_id']
# train.drop(['transaction_id'],axis=1,inplace=True)
# test.drop(['transaction_id'],axis=1,inplace=True)

In [523]:
train['cat_var_1'].fillna('gf',inplace=True)
test['cat_var_1'].fillna('gf',inplace=True)
train['cat_var_3'].fillna('qt',inplace=True)
test['cat_var_3'].fillna('qt',inplace=True)
test['cat_var_6'].fillna('zs',inplace=True)
train['cat_var_8'].fillna('dn',inplace=True)
test['cat_var_8'].fillna('dn',inplace=True)

In [524]:
# ## merge 19 and 20
# train.drop(['cat_var_20'],axis=1,inplace=True)
# test.drop(['cat_var_20'],axis=1,inplace=True)

In [525]:
# ## merge 21,22,23
# train['cat_var_21-23']=train.loc[:,['cat_var_22','cat_var_23']].sum(axis=1)
# test['cat_var_21-23']=test.loc[:,['cat_var_22','cat_var_23']].sum(axis=1)

# train.drop(['cat_var_21','cat_var_22','cat_var_23'],axis=1,inplace=True)
# test.drop(['cat_var_21','cat_var_22','cat_var_23'],axis=1,inplace=True)

In [526]:
# ## merge 24-42
# train['cat_var_24-42']=train.loc[:,list(train.columns[27:45])].sum(axis=1)
# test['cat_var_24-42']=test.loc[:,list(test.columns[27:45])].sum(axis=1)

# train.drop(list(train.columns[27:45]),axis=1,inplace=True)
# test.drop(list(test.columns[27:45]),axis=1,inplace=True)

In [527]:
# train.drop(['cat_var_24'],axis=1,inplace=True)
# test.drop(['cat_var_24'],axis=1,inplace=True)

In [528]:
# li=[]
# for index,row in test.iterrows():
#     if (row['cat_var_1'] in ['jk', 'gz', 'yw']) or (row['cat_var_3'] in ['ex', 'ye', 'hr', 'ap']) or (row['cat_var_8'] in ['da', 'db', 'mh']):
#         li.append(True)
#     else: li.append(False)

In [529]:
# dic=dict(df['cat_var_21-23'].value_counts())
# df['var_21-23_counts']=df['cat_var_21-23'].apply(lambda x: dic[x])
# df.drop(['cat_var_21-23'],axis=1,inplace=True)

In [530]:
# dic=dict(df['cat_var_24-42'].value_counts())
# df['var_24-42_counts']=df['cat_var_24-42'].apply(lambda x: dic[x])
# df.drop(['cat_var_24-42'],axis=1,inplace=True)

In [531]:
df=pd.concat([train,test])
for i in range(1,19):
    dic=dict(df['cat_var_%d'%i].value_counts())
    df['var_%d_counts'%i]=df['cat_var_%d'%i].apply(lambda x: dic[x])
    df.drop(['cat_var_%d'%i],axis=1,inplace=True)

In [532]:
train=df.iloc[:len(train),:]
test=df.iloc[len(train):,:]

In [533]:
print len(train),len(test)

348978 523466


In [552]:
trainX.head()

,cat_var_19,cat_var_20,cat_var_21,cat_var_22,cat_var_23,cat_var_24,cat_var_25,cat_var_26,cat_var_27,cat_var_28,cat_var_29,cat_var_30,cat_var_31,cat_var_32,cat_var_33,cat_var_34,cat_var_35,cat_var_36,cat_var_37,cat_var_38,cat_var_39,cat_var_40,cat_var_41,cat_var_42,num_var_1,num_var_2,num_var_3,num_var_4,num_var_5,num_var_6,num_var_7,target,transaction_id,var_1_counts,var_2_counts,var_3_counts,var_4_counts,var_5_counts,var_6_counts,var_7_counts,var_9_counts,var_10_counts,var_11_counts,var_12_counts,var_13_counts,var_14_counts,var_15_counts,var_16_counts,var_17_counts,var_18_counts
0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.302632e-08,0.040182,0.0,1.800000e-07,2.302632e-08,2.368421e-08,1.115205e-08,0.0,id_11,365237,582056,20117,289199,436279,14198,871363,183011,26768,190308,81837,22321,620874,590455,801005,634373,497666
1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7.965789e-06,0.157872,0.0,2.105000e-06,2.769737e-07,7.965789e-06,2.433058e-06,0.0,id_33,253029,258183,8585,583245,436165,252763,871363,168244,32032,190308,296803,261510,620874,281989,801005,634373,374778
2,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7.828947e-08,0.089140,0.0,3.550000e-07,4.671053e-08,1.052632e-07,4.276014e-07,0.0,id_51,365237,582056,3605,583245,436165,352425,871363,168244,80714,205398,357150,332353,620874,590455,801005,634373,497666
3,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7.894737e-08,0.227239,0.0,1.050000e-06,1.381579e-07,2.190789e-07,1.848054e-08,0.0,id_54,365237,582056,2354,289199,436165,4955,871363,204002,26156,190308,81837,14004,29756,590455,801005,634373,497666
4,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.321053e-06,0.160410,0.0,2.105000e-06,2.769737e-07,3.340789e-06,2.152983e-06,0.0,id_62,253029,258183,260,583245,436165,252763,871363,204002,43239,196216,296803,261510,620874,281989,801005,634373,374778


In [670]:
trainX=train.copy(deep=True)
testX=test.copy(deep=True)

In [671]:
trainX.var()

cat_var_19       2.495895e-01
cat_var_20       2.495895e-01
cat_var_21       2.203281e-01
cat_var_22       2.182442e-01
cat_var_23       5.910596e-03
cat_var_24       9.018235e-04
cat_var_25       1.232021e-04
cat_var_26       5.527392e-04
cat_var_27       2.865510e-06
cat_var_28       2.578900e-05
cat_var_29       5.444188e-05
cat_var_30       1.174724e-04
cat_var_31       0.000000e+00
cat_var_32       5.731004e-06
cat_var_33       5.731004e-06
cat_var_34       2.865510e-06
cat_var_35       0.000000e+00
cat_var_36       0.000000e+00
cat_var_37       0.000000e+00
cat_var_38       0.000000e+00
cat_var_39       8.596481e-06
cat_var_40       0.000000e+00
cat_var_41       2.865510e-06
cat_var_42       0.000000e+00
num_var_1        3.728559e-06
num_var_2        1.729197e-02
num_var_3        6.443041e-06
num_var_4        3.999790e-06
num_var_5        5.203798e-07
num_var_6        2.229019e-06
num_var_7        2.137943e-06
target           9.561288e-02
var_1_counts     2.165901e+10
var_2_coun

In [672]:
testX.var()

cat_var_19       2.494947e-01
cat_var_20       2.494947e-01
cat_var_21       2.204461e-01
cat_var_22       2.183211e-01
cat_var_23       6.036113e-03
cat_var_24       8.989640e-04
cat_var_25       1.012382e-04
cat_var_26       5.842246e-04
cat_var_27       5.731009e-06
cat_var_28       2.292364e-05
cat_var_29       6.112738e-05
cat_var_30       7.640806e-05
cat_var_31       5.731009e-06
cat_var_32       9.551646e-06
cat_var_33       7.641331e-06
cat_var_34       5.731009e-06
cat_var_35       5.731009e-06
cat_var_36       1.910344e-06
cat_var_37       1.910344e-06
cat_var_38       0.000000e+00
cat_var_39       5.731009e-06
cat_var_40       3.820680e-06
cat_var_41       0.000000e+00
cat_var_42       0.000000e+00
num_var_1        2.700855e-06
num_var_2        1.734112e-02
num_var_3        1.476668e-05
num_var_4        7.248973e-06
num_var_5        2.677154e-07
num_var_6        2.374646e-06
num_var_7        4.974097e-06
target                    NaN
var_1_counts     2.180484e+10
var_2_coun

In [660]:
trainX.drop(['var_8_counts'],axis=1,inplace=True)
testX.drop(['var_8_counts'],axis=1,inplace=True)

In [661]:
trainX.drop(['var_6_counts'],axis=1,inplace=True)
testX.drop(['var_6_counts'],axis=1,inplace=True)

In [662]:
# trainX.drop(['var_4_counts'],axis=1,inplace=True)
# testX.drop(['var_4_counts'],axis=1,inplace=True)

In [663]:
predictors=[x for x in trainX.columns if x not in ['target','transaction_id']]
target=['target']

In [664]:
# X_train,X_test,y_train,y_test=train_test_split(trainX[predictors].values,trainX[target].values,test_size=0.3)

In [665]:
def run(X,y,test,params):
    dtrain=xgb.DMatrix(X,label=y)
    dtest=xgb.DMatrix(test)
    bst=xgb.train(params,dtrain,99)
    preds=bst.predict(dtest)
    return bst,preds

params={
    'objective':'binary:logistic',
    'eta':0.11,
    'max_depth':9,
#     'min_child_weight':4,
#     'gamma':0.1,
#     'subsample':0.8,
#     'colsample_bytree':0.7,
#     'scale_pos_weight':1,
#     'reg_alpha':1e-05
}

In [666]:
# bst1,p1=run(X_train,y_train,X_test,params)
# print roc_auc_score(y_test,p1)
# p2=(p1>0.5).astype('int')
# print accuracy_score(y_test,p2)
# print confusion_matrix(y_test,p2)

In [667]:
bst,xgpreds=run(trainX[predictors].values,trainX[target].values,testX[predictors].values,params)

In [676]:
subs=pd.DataFrame({
    'transaction_id':testX['transaction_id'],
    'target':f
},columns=['transaction_id','target'])

In [677]:
subs.to_csv('/Users/shashank/Downloads/Societe2/%d.csv'%i,index=False)
i=i+1

In [506]:
param2 = {
    'objective':'binary',
    'max_depth':9,
    'min_data_in_leaf':25,
    'learning_rate':0.11,
    }
param2['metric'] = ['auc', 'binary_logloss']
num_round=155



In [507]:
lgbm=lgb.train(param2,train_data,num_round)


In [508]:
ypred2=lgbm.predict(testX[predictors].values)

In [417]:
ypred2

array([ 0.06121049,  0.0590109 ,  0.06589795, ...,  0.06334938,
        0.0672889 ,  0.07002755])

In [369]:
# param2 = {
#     'num_leaves':31,
#     'objective':'binary',
#     'max_depth':9,
#     'learning_rate':0.11,
#     }
# param2['metric'] = ['auc', 'binary_logloss']
# num_round=155



array([ 0.05894795,  0.06829389,  0.06575692, ...,  0.06636461,
        0.06750999,  0.05794487], dtype=float32)

In [673]:
high=pd.read_csv('/Users/shashank/Downloads/7363.csv')
second=pd.read_csv('/Users/shashank/Downloads/Societe2/106.csv')

In [674]:
f=0.5*high['target']+0.5*high['target']

In [675]:
pearsonr(high['target'],second['target'])

(0.99146176676344777, 0.0)

In [369]:
# ---45 second high --73629
# ---47 highest -- 0.73683	ensemble of 45 and 73630
## -- 67 -- 0.73653 with eta 0.11 depth 9 and trees=99
## --68 120 tress 73594
### 69 -- 0.73602 108
### 70 -- 0.73609 104 trees
### 71 --- 0.73640 102 trees
### 72 --- 0.73650 100 trees
### 83-- cat_var_9 from var_count_9 to one hot --0.73519 so var count worked for a feature with 5 unique labels
## 84 -- sice var_counts is working (from 83) cat_var_21-23 is converted into var_counts--0.73495 
## 85 -- best without var_counts_9--0.0.73582	
##86 -- best without var_counts_4==0.73465

In [330]:
trainX.head()

,cat_var_19,cat_var_20,cat_var_21,cat_var_22,cat_var_23,cat_var_24,cat_var_25,cat_var_26,cat_var_27,cat_var_28,cat_var_29,cat_var_30,cat_var_31,cat_var_32,cat_var_33,cat_var_34,cat_var_35,cat_var_36,cat_var_37,cat_var_38,cat_var_39,cat_var_40,cat_var_41,cat_var_42,num_var_1,num_var_2,num_var_3,num_var_4,num_var_5,num_var_6,num_var_7,target,transaction_id,var_1_counts,var_2_counts,var_3_counts,var_5_counts,var_6_counts,var_7_counts,var_8_counts,var_9_counts,var_10_counts,var_11_counts,var_12_counts,var_13_counts,var_14_counts,var_15_counts,var_16_counts,var_17_counts,var_18_counts
0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.302632e-08,0.040182,0.0,1.800000e-07,2.302632e-08,2.368421e-08,1.115205e-08,0.0,id_11,365237,582056,20117,436279,14198,871363,448639,183011,26768,190308,81837,22321,620874,590455,801005,634373,497666
1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7.965789e-06,0.157872,0.0,2.105000e-06,2.769737e-07,7.965789e-06,2.433058e-06,0.0,id_33,253029,258183,8585,436165,252763,871363,448639,168244,32032,190308,296803,261510,620874,281989,801005,634373,374778
2,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7.828947e-08,0.089140,0.0,3.550000e-07,4.671053e-08,1.052632e-07,4.276014e-07,0.0,id_51,365237,582056,3605,436165,352425,871363,448639,168244,80714,205398,357150,332353,620874,590455,801005,634373,497666
3,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7.894737e-08,0.227239,0.0,1.050000e-06,1.381579e-07,2.190789e-07,1.848054e-08,0.0,id_54,365237,582056,2354,436165,4955,871363,17438,204002,26156,190308,81837,14004,29756,590455,801005,634373,497666
4,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.321053e-06,0.160410,0.0,2.105000e-06,2.769737e-07,3.340789e-06,2.152983e-06,0.0,id_62,253029,258183,260,436165,252763,871363,448639,204002,43239,196216,296803,261510,620874,281989,801005,634373,374778


In [520]:
trainX.var()

cat_var_19       2.495895e-01
cat_var_20       2.495895e-01
cat_var_21       2.203281e-01
cat_var_22       2.182442e-01
cat_var_23       5.910596e-03
cat_var_24       9.018235e-04
cat_var_25       1.232021e-04
cat_var_26       5.527392e-04
cat_var_27       2.865510e-06
cat_var_28       2.578900e-05
cat_var_29       5.444188e-05
cat_var_30       1.174724e-04
cat_var_31       0.000000e+00
cat_var_32       5.731004e-06
cat_var_33       5.731004e-06
cat_var_34       2.865510e-06
cat_var_35       0.000000e+00
cat_var_36       0.000000e+00
cat_var_37       0.000000e+00
cat_var_38       0.000000e+00
cat_var_39       8.596481e-06
cat_var_40       0.000000e+00
cat_var_41       2.865510e-06
cat_var_42       0.000000e+00
num_var_1        3.728559e-06
num_var_2        1.729197e-02
num_var_3        6.443041e-06
num_var_4        3.999790e-06
num_var_5        5.203798e-07
num_var_6        2.229019e-06
num_var_7        2.137943e-06
target           9.561288e-02
var_1_counts     2.165901e+10
var_2_coun

In [518]:
np.power(train.num_var_1,10000000)

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
5         0.0
6         0.0
7         0.0
8         0.0
9         0.0
10        0.0
11        0.0
12        0.0
13        0.0
14        0.0
15        0.0
16        0.0
17        0.0
18        0.0
19        0.0
20        0.0
21        0.0
22        0.0
23        0.0
24        0.0
25        0.0
26        0.0
27        0.0
28        0.0
29        0.0
         ... 
348948    0.0
348949    0.0
348950    0.0
348951    0.0
348952    0.0
348953    0.0
348954    0.0
348955    0.0
348956    0.0
348957    0.0
348958    0.0
348959    0.0
348960    0.0
348961    0.0
348962    0.0
348963    0.0
348964    0.0
348965    0.0
348966    0.0
348967    0.0
348968    0.0
348969    0.0
348970    0.0
348971    0.0
348972    0.0
348973    0.0
348974    0.0
348975    0.0
348976    0.0
348977    0.0
Name: num_var_1, Length: 348978, dtype: float64

In [290]:
a=trainX.loc[:,['cat_var_21','cat_var_22','cat_var_23']].values
b=testX.loc[:,['cat_var_21','cat_var_22','cat_var_23']].values

In [291]:
c=[np.where(r==1)[0][0] for r in a ]
d=[np.where(r==1)[0][0] for r in b ]

In [292]:
pd.Series(c).value_counts()

0    234603
1    112300
2      2075
dtype: int64

In [293]:
trainX['cat_var_21-23']=c
testX['cat_var_21-23']=d


In [294]:
trainX.drop(['cat_var_21','cat_var_22','cat_var_23'],axis=1,inplace=True)
testX.drop(['cat_var_21','cat_var_22','cat_var_23'],axis=1,inplace=True)

In [138]:
def to_popularity(col,train,test):
    df=pd.concat([train,test])
    dic=dict(df[col].value_counts())
    df['popularity_%s'%col]=df[col].apply(lambda x: dic[x])
    df.drop([col],axis=1,inplace=True)
    train=df.iloc[:len(train),:]
    test=df.iloc[len(train):,:]
    return train,test

In [295]:
trainX,testX=to_popularity('cat_var_21-23',trainX,testX)
# trainX,testX=to_popularity('cat_var_20',trainX,testX)

In [201]:
trainX,testX=to_popularity('cat_var_21',trainX,testX)
trainX,testX=to_popularity('cat_var_22',trainX,testX)
trainX,testX=to_popularity('cat_var_23',trainX,testX)

In [253]:
c=pd.get_dummies(pd.concat([trainX,testX]),columns=['cat_var_9'])

In [254]:
trainX=c.iloc[:len(train),:]
testX=c.iloc[len(train):,:]

In [296]:
trainX

,cat_var_19,cat_var_20,cat_var_24,cat_var_25,cat_var_26,cat_var_27,cat_var_28,cat_var_29,cat_var_30,cat_var_31,cat_var_32,cat_var_33,cat_var_34,cat_var_35,cat_var_36,cat_var_37,cat_var_38,cat_var_39,cat_var_40,cat_var_41,cat_var_42,num_var_1,num_var_2,num_var_3,num_var_4,num_var_5,num_var_6,num_var_7,target,transaction_id,var_1_counts,var_2_counts,var_3_counts,var_4_counts,var_5_counts,var_6_counts,var_7_counts,var_9_counts,var_10_counts,var_11_counts,var_12_counts,var_13_counts,var_14_counts,var_15_counts,var_16_counts,var_17_counts,var_18_counts,popularity_cat_var_21-23
0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.302632e-08,0.040182,0.0,1.800000e-07,2.302632e-08,2.368421e-08,1.115205e-08,0.0,id_11,365237,582056,20117,289199,436279,14198,871363,183011,26768,190308,81837,22321,620874,590455,801005,634373,497666,586327
1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7.965789e-06,0.157872,0.0,2.105000e-06,2.769737e-07,7.965789e-06,2.433058e-06,0.0,id_33,253029,258183,8585,583245,436165,252763,871363,168244,32032,190308,296803,261510,620874,281989,801005,634373,374778,586327
2,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7.828947e-08,0.089140,0.0,3.550000e-07,4.671053e-08,1.052632e-07,4.276014e-07,0.0,id_51,365237,582056,3605,583245,436165,352425,871363,168244,80714,205398,357150,332353,620874,590455,801005,634373,497666,586327
3,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7.894737e-08,0.227239,0.0,1.050000e-06,1.381579e-07,2.190789e-07,1.848054e-08,0.0,id_54,365237,582056,2354,289199,436165,4955,871363,204002,26156,190308,81837,14004,29756,590455,801005,634373,497666,586327
4,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.321053e-06,0.160410,0.0,2.105000e-06,2.769737e-07,3.340789e-06,2.152983e-06,0.0,id_62,253029,258183,260,583245,436165,252763,871363,204002,43239,196216,296803,261510,620874,281989,801005,634373,374778,586327
5,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.953947e-07,0.078566,0.0,3.550000e-07,4.671053e-08,4.407895e-08,9.463310e-08,0.0,id_67,365237,582056,184452,583245,436279,352425,871363,183011,77365,201511,357150,332353,620874,590455,801005,634373,497666,280863
6,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.736842e-07,0.600508,0.0,4.575000e-06,5.986842e-07,1.736842e-07,8.411831e-08,0.0,id_71,1796,582056,184452,289199,436165,1789,871363,142171,77365,201511,357150,2568,33173,590455,801005,238071,374778,586327
7,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.789474e-08,0.152374,0.0,2.105000e-06,2.769737e-07,5.789474e-08,2.453451e-08,0.0,id_88,253029,258183,2292,583245,436165,252763,871363,175016,28859,201511,296803,261510,620874,281989,801005,238071,374778,586327
8,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.289474e-08,0.093053,0.0,7.500000e-07,8.223684e-08,9.868421e-08,1.274520e-08,0.0,id_95,49788,582056,184452,289199,436279,2386,871363,183011,38575,190308,72522,53281,67241,590455,71439,634373,374778,586327
9,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.500000e-07,0.158401,0.0,2.105000e-06,2.769737e-07,5.500000e-07,3.138505e-07,0.0,id_97,253029,258183,3308,583245,436165,252763,871363,183011,32032,196216,296803,261510,620874,281989,801005,238071,374778,586327


In [199]:
trainX.drop(['popularity_cat_var_20'],axis=1,inplace=True)
testX.drop(['popularity_cat_var_20'],axis=1,inplace=True)

In [331]:
predictors

['cat_var_19',
 'cat_var_20',
 'cat_var_21',
 'cat_var_22',
 'cat_var_23',
 'cat_var_24',
 'cat_var_25',
 'cat_var_26',
 'cat_var_27',
 'cat_var_28',
 'cat_var_29',
 'cat_var_30',
 'cat_var_31',
 'cat_var_32',
 'cat_var_33',
 'cat_var_34',
 'cat_var_35',
 'cat_var_36',
 'cat_var_37',
 'cat_var_38',
 'cat_var_39',
 'cat_var_40',
 'cat_var_41',
 'cat_var_42',
 'num_var_1',
 'num_var_2',
 'num_var_3',
 'num_var_4',
 'num_var_5',
 'num_var_6',
 'num_var_7',
 'var_1_counts',
 'var_2_counts',
 'var_3_counts',
 'var_5_counts',
 'var_6_counts',
 'var_7_counts',
 'var_8_counts',
 'var_9_counts',
 'var_10_counts',
 'var_11_counts',
 'var_12_counts',
 'var_13_counts',
 'var_14_counts',
 'var_15_counts',
 'var_16_counts',
 'var_17_counts',
 'var_18_counts']

In [ ]:
### 101 --0.73522 18 removed
### 102 -- 0.73424 17 removed
### 103 -- 0.73557 16 removed